# Direct sampling with Gaussian process regression

#### Import general modules

mpi4py is always required when using these tools. Numpy is always good to have if any manipulation is to be done.

In [19]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

#### Import modules from pynektools

In this case we will import all the data types that we currently support, as well as io functions that are required to populate them.

In [20]:
# Data types
from pynektools.datatypes.msh import Mesh
from pynektools.datatypes.coef import Coef
from pynektools.datatypes.field import Field, FieldRegistry

# Readers
from pynektools.io.ppymech.neksuite import preadnek, pynekread

# Writers
from pynektools.io.ppymech.neksuite import pwritenek, pynekwrite

fname = '/home/adperez/cpc_gaussian_process_compression/data/mixlay/mixlay0.f00001'

## Read the data

In [21]:
msh = Mesh(comm, create_connectivity=False)
fld = FieldRegistry(comm)

pynekread(fname, comm, data_dtype=np.double, msh = msh, fld = fld)


for e in range(msh.nelv):
    if (np.min(msh.x[e]), np.max(msh.x[e])) == (0, 0.25):
        if (np.min(msh.y[e]), np.max(msh.y[e])) == (0, 1.1399999856948853):
            print(e)
    
coef = Coef(msh=msh, comm=comm)

2025-02-19 18:12:26,351 - Mesh - INFO - Initializing empty Mesh object.
2025-02-19 18:12:26,353 - Field - INFO - Initializing empty Field object
2025-02-19 18:12:26,354 - pynekread - INFO - Reading file: /home/adperez/cpc_gaussian_process_compression/data/mixlay/mixlay0.f00001
2025-02-19 18:12:26,357 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2025-02-19 18:12:26,358 - Mesh - INFO - Initializing common attributes.
2025-02-19 18:12:26,359 - Mesh - INFO - Getting vertices
2025-02-19 18:12:26,360 - Mesh - INFO - Getting edge centers
2025-02-19 18:12:26,367 - Mesh - INFO - Facet centers not available for 2D
2025-02-19 18:12:26,368 - Mesh - INFO - Mesh object initialized.
2025-02-19 18:12:26,370 - Mesh - INFO - Mesh data is of type: float64
2025-02-19 18:12:26,370 - Mesh - INFO - Elapsed time: 0.013423200999994833s
2025-02-19 18:12:26,371 - pynekread - INFO - Reading field data
2025-02-19 18:12:26,373 - pynekread - INFO - File read
2025-02-19 18:12:26,374 - pynekread - INF

## Initialize the Direct sampler

In [22]:
from pynektools.compression.gpc_direct_sampling import DirectSampler

ds = DirectSampler(comm=comm, msh=msh)

In [ ]:
#ds.estimate_covariance("average", elements_to_average=100)
#ds.sample(method="fixed_bitrate", n_samples=63)

ds.sample_field(field=fld.registry["u"], field_name="u", covariance_method="average", covariance_elements_to_average=100, compression_method="fixed_bitrate", bitrate = 0.25)


2025-02-19 18:12:26,463 - DirectSampler - INFO - Sampling the field with options: covariance_method: {covariance_method}, compression_method: {compression_method}
2025-02-19 18:12:26,464 - DirectSampler - INFO - Estimating the covariance matrix
2025-02-19 18:12:26,466 - DirectSampler - INFO - Transforming the field into to legendre space
2025-02-19 18:12:26,473 - DirectSampler - INFO - Estimating the covariance matrix using the averaging method method. Averaging over 100 elements at a time
2025-02-19 18:12:26,475 - DirectSampler - INFO - Covariance saved in field data_to_compress["u"]["kw"]
2025-02-19 18:12:26,478 - DirectSampler - INFO - Sampling the field using the fixed bitrate method. using settings: {'method': 'fixed_bitrate', 'bitrate': 0.1, 'n_samples': 7}
2025-02-19 18:12:26,878 - DirectSampler - INFO - Sampled_field saved in field data_to_compress["u"]["field"]


In [24]:
# Hack it a bit to make the rest work

ds.field = fld.registry["u"]
ds.field_sampled = ds.data_to_compress["u"]["field"]
ds.kw = ds.data_to_compress["u"]["kw"]
ds.elements_to_average = ds.settings["covariance"]["elements_to_average"]

In [25]:
ind_50 = np.where(ds.field_sampled[0] == -50)
#print(ds.field[0])
#print(ds.field_sampled[0])
a = abs(ds.field_sampled[0])-abs(ds.field[0])
a[ind_50] = -50
print(a)

[[[  0. -50. -50. -50. -50. -50. -50.   0.]
  [-50. -50. -50. -50. -50. -50. -50. -50.]
  [-50. -50. -50. -50. -50. -50. -50. -50.]
  [-50. -50. -50. -50. -50. -50. -50. -50.]
  [-50. -50. -50. -50. -50. -50. -50. -50.]
  [-50. -50. -50.   0. -50. -50. -50. -50.]
  [-50.   0. -50. -50.   0. -50. -50. -50.]
  [  0. -50. -50. -50. -50. -50. -50.   0.]]]


In [26]:

print(ds.field_hat[1466].reshape(-1,1, order='F'))

print("======")

print(ds.field_sampled[1466].reshape(-1,1, order='F'))

[[ 1.99990441e-01]
 [-5.57531503e-06]
 [-1.80462288e-08]
 [ 7.00211732e-09]
 [-9.51879178e-09]
 [ 2.13566155e-09]
 [-5.95115817e-10]
 [-5.36121616e-10]
 [-8.59488645e-06]
 [-5.00078505e-06]
 [-3.83074439e-09]
 [ 8.52690949e-09]
 [-8.40598942e-09]
 [ 2.39046847e-09]
 [-8.58811016e-10]
 [-2.42388614e-10]
 [-2.37776126e-06]
 [-1.36844321e-06]
 [ 1.57842481e-08]
 [ 5.69112633e-09]
 [-2.72724125e-09]
 [ 4.27633113e-10]
 [-4.92429909e-10]
 [ 4.96550225e-10]
 [ 1.13080070e-07]
 [ 7.57060759e-08]
 [ 1.00539981e-08]
 [ 8.09318145e-10]
 [-2.86645941e-10]
 [ 2.32829433e-10]
 [ 1.42264199e-10]
 [ 4.43830162e-10]
 [ 8.43976818e-08]
 [ 4.96585829e-08]
 [-1.09479314e-09]
 [ 8.54305353e-10]
 [ 9.55948182e-10]
 [-8.39331422e-10]
 [ 3.69644683e-10]
 [ 2.50419075e-10]
 [-1.66313403e-08]
 [-1.06633414e-08]
 [-2.55425789e-09]
 [-9.41682210e-11]
 [ 1.41116511e-09]
 [-6.89430439e-10]
 [ 6.21008604e-11]
 [ 1.17915297e-09]
 [-4.47323203e-09]
 [-2.63863014e-09]
 [-1.17943747e-09]
 [-1.12387309e-09]
 [ 1.3693124

In [27]:
recon = ds.predict(ds.field_sampled)

AttributeError: 'DirectSampler' object has no attribute 'transformed_index'

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,fld.registry["u"].flatten(), levels=100, cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
#c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() , fld.registry["u"].flatten() - recon.flatten(), levels=100, cmap="RdBu_r")
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,recon.flatten(), levels=100, cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()

In [ ]:
print(np.mean(fld.registry["u"] - recon))